이를 위해 pyspark과 Py4J 패키지를 설치한다. Py4J 패키지는 파이썬 프로그램이 자바가상머신상의 오브젝트들을 접근할 수 있게 해준다. Local Standalone Spark을 사용한다.

In [ ]:
!pip install pyspark==3.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=7b0349f333d34615658819895c2ff360d652e0e3a24de05d35ab9c434c725e43
  Stored in directory: /root/.cache/pip/wheels/97/f5/c0/947e2c0942b361ffe58651f36bd7f13772675b3863fd63d1b1
Successfully built pyspark
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.4
    Uninstalling pyspark-3.5.4:
      Successfully uninstalled pyspark-3.5.4


In [ ]:
!wget https://s3-geospatial.s3.us-west-2.amazonaws.com/orders.csv

--2025-02-10 08:03:41--  https://s3-geospatial.s3.us-west-2.amazonaws.com/orders.csv
Resolving s3-geospatial.s3.us-west-2.amazonaws.com (s3-geospatial.s3.us-west-2.amazonaws.com)... 52.92.162.138, 52.92.248.58, 52.92.176.130, ...
Connecting to s3-geospatial.s3.us-west-2.amazonaws.com (s3-geospatial.s3.us-west-2.amazonaws.com)|52.92.162.138|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89951 (88K) [text/csv]
Saving to: ‘orders.csv’

orders.csv          100%[===================>]  87.84K  --.-KB/s    in 0.1s    

2025-02-10 08:03:42 (620 KB/s) - ‘orders.csv’ saved [89951/89951]



**Spark Session**

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark Hive") \
    .enableHiveSupport() \
    .getOrCreate()

In [ ]:
!ls -tl

total 92
drwxr-xr-x 1 root root  4096 Feb  6 14:19 sample_data
-rw-r--r-- 1 root root 89951 Apr 24  2022 orders.csv


In [ ]:
# Read in data and create a DataFrame
df = spark.read.csv("orders.csv", inferSchema=True, header=True, sep ='\t')

In [ ]:
df.show(5)

+------------+--------------------+
|    order_id|               items|
+------------+--------------------+
|860196503764|[{"name": "DAILY ...|
|860292645076|[{"name": "DAILY ...|
|860320956628|[{"name": "DAILY ...|
|860321513684|[{"name": "DAILY ...|
|862930665684|[{"name": "DAILY ...|
+------------+--------------------+
only showing top 5 rows



In [ ]:
spark.sql("CREATE DATABASE IF NOT EXISTS TEST_DB")
spark.sql("USE TEST_DB")

DataFrame[]

In [ ]:
spark.sql("SHOW DATABASES").show()

+---------+
|namespace|
+---------+
|  default|
|  test_db|
+---------+



In [ ]:
!ls -tl

total 104
drwxr-xr-x 3 root root  4096 Feb 10 08:04 spark-warehouse
drwxr-xr-x 5 root root  4096 Feb 10 08:04 metastore_db
-rw-r--r-- 1 root root   669 Feb 10 08:04 derby.log
drwxr-xr-x 1 root root  4096 Feb  6 14:19 sample_data
-rw-r--r-- 1 root root 89951 Apr 24  2022 orders.csv


In [ ]:
df.write.saveAsTable("TEST_DB.orders", mode="overwrite")

In [ ]:
!ls -tl spark-warehouse/test_db.db/orders/

total 24
-rw-r--r-- 1 root root     0 Feb 10 08:05 _SUCCESS
-rw-r--r-- 1 root root 23051 Feb 10 08:05 part-00000-2ce5f349-4535-45ab-a638-11c754b3e307-c000.snappy.parquet


In [ ]:
spark.sql("SELECT * FROM TEST_DB.orders").show(5)

+------------+--------------------+
|    order_id|               items|
+------------+--------------------+
|860196503764|[{"name": "DAILY ...|
|860292645076|[{"name": "DAILY ...|
|860320956628|[{"name": "DAILY ...|
|860321513684|[{"name": "DAILY ...|
|862930665684|[{"name": "DAILY ...|
+------------+--------------------+
only showing top 5 rows



In [ ]:
spark.table("TEST_DB.orders").show(5)

+------------+--------------------+
|    order_id|               items|
+------------+--------------------+
|860196503764|[{"name": "DAILY ...|
|860292645076|[{"name": "DAILY ...|
|860320956628|[{"name": "DAILY ...|
|860321513684|[{"name": "DAILY ...|
|862930665684|[{"name": "DAILY ...|
+------------+--------------------+
only showing top 5 rows



In [ ]:
!ls -tl metastore_db/

total 28
drwxr-xr-x 2 root root 4096 Feb 10 08:05 seg0
-rw-r--r-- 1 root root  900 Feb 10 08:04 service.properties
-rw-r--r-- 1 root root  608 Feb 10 08:04 README_DO_NOT_TOUCH_FILES.txt
drwxr-xr-x 2 root root 4096 Feb 10 08:04 log
-rw-r--r-- 1 root root   38 Feb 10 08:04 db.lck
-rw-r--r-- 1 root root    4 Feb 10 08:04 dbex.lck
drwxr-xr-x 2 root root 4096 Feb 10 08:04 tmp


In [ ]:
spark.catalog.listTables()

[Table(name='orders', catalog='spark_catalog', namespace=['test_db'], description=None, tableType='MANAGED', isTemporary=False)]

In [ ]:
spark.sql("""
    DROP TABLE IF EXISTS TEST_DB.orders_count;
""")

DataFrame[]

In [ ]:
spark.sql("""
    CREATE TABLE TEST_DB.orders_count AS
    SELECT order_id, COUNT(1) as count
    FROM TEST_DB.orders
    GROUP BY 1""")

DataFrame[]

In [ ]:
spark.catalog.listTables()

[Table(name='orders', catalog='spark_catalog', namespace=['test_db'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='orders_count', catalog='spark_catalog', namespace=['test_db'], description=None, tableType='MANAGED', isTemporary=False)]

In [ ]:
!ls -tl spark-warehouse/test_db.db/

total 8
drwxr-xr-x 2 root root 4096 Feb 10 08:06 orders_count
drwxr-xr-x 2 root root 4096 Feb 10 08:05 orders


In [ ]:
spark.sql("SELECT * FROM test_db.orders_count LIMIT 10").show()

+-------------+-----+
|     order_id|count|
+-------------+-----+
|1209310019796|    1|
|1253925257428|    1|
|1314797846740|    1|
|1370637402324|    1|
|1967690285268|    1|
|1971226443988|    1|
|2106987970772|    1|
|2135281533140|    1|
|1271771070676|    1|
|1713331765460|    1|
+-------------+-----+

